'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
'''

In [1]:
'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#NOW DEFINE DB SCHEMA (THIS DOESN'T WRITE SCHEMA TO DB, JUST TO SQLALCHEMY CLASSES AND OBJECTS)
#define an SQLAlchemy base class to maintain catalog of classes and tables relative to this base
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create tables

import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import expression as Expr
import importSpecial as importSpecial



In [3]:
 myTable = Base.metadata.tables['expressions']

In [4]:
'''
Dictionary of "SQLAlchemy where clause lambda functions" that tests record uniqueness.
e.g. entries in the 'name' field of the people table must be unique.
Therefore, we determine whether to insert/update a csv row into the people table based on
whether the csv row's name field value exists in the people table.

if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'people': lambda RowVal: Base.metadata.tables['people'].c['name'] == RowVal['name'],
    'locations': False
}

# importCSV('testlab\\2_table_input.csv', unqTests)
# #test some reords
# for u, a in session.query(People, Locations).filter(People.id==Locations.people_id):
#     print (u,a)
    
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') #import feasibillity questions, build feasibility expressions


session.commit()

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist

Reading csv record: Feas-2
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-3
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-4
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-5
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-6
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-7
Adding to variable dictionary: Count_CB

Reading csv record: Feas-8
Adding to variable dictionary: Runoff_Type

Reading csv record: Feas-9
Adding to variable dictionary: TFMR_Exist

Reading csv record: Feas-10
Adding to variable dictionary: DS_SS_Exist

Reading csv record: Feas-11
Adding to variable dictionary: Fac_Slope

Reading csv record: Feas-12
Adding to variable dictionary: Can_Add_SD

Reading csv record: Feas-13
Adding to variable dictionary: Pave_Area
Adding to variable dictionary: BMP_Size(bas

In [5]:
#### build base bmp importer:
# functions to help with determining function status
# uses funcStatus list:
#func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
#                                         [1] = fault description
def isFuncStatusOK (funcStatus):
    #return 0 if fault, 1 if no fault
    return funcStatus[0]

def getFuncStatusFault (funcStatus):
    return funcStatus[1]
def _HELPER_ImportBaseBMPsCSV_Expr (importLS, row, csvHeadersLS):
    import expression as Expr #expression inserting module
    #import expression and variable at current row.
    #call procInputVarDecs, for current record at CSV column var_unit_ColName
    #if no error then register the expr_name expression uwing information at CSV column expr_ColName
    #return imported expression_id
    
    '''args:
        importLS: information needed for import. elements includes:
                    [UnitCSVColumnName, ExpressionName, ExprStrCSVColumnName, statusStatment]
        row: csv row that we want to import expression from
        csvHeadersLS: list of header names for each csv column. ORDER ASSUMED TO MATCH ROW
    '''
    VarDict = {} #init empty VarDict for expression
    #proce variable declaration. assume variable's value can be found in the facility_chars table
    retStatus = Expr.procInputVarDecs(VarDict, row[csvHeadersLS.index(importLS[0])], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
    if isFuncStatusOK(retStatus[0]) == 0: #check if return status ok
        print (getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
        my_expr_id=-1234
    else:              
        my_expr_id = Expr.registerExpr (importLS[1],  row[csvHeadersLS.index(importLS[2])], VarDict)
    

def importBaseBMPsCSV(importFilePath):
    import csv
    import expression as Expr #expression inserting module
    #define csv column constants:
    C_feastest_StartCol = 19 #1st column holding feasibility test. assume remaining rows are all feasibility tests, identified by column name 
    print ('Reading csv for import to base bmp tables')
    with open(importFilePath, 'rt', encoding='utf-8-sig') as csvfile:
        csvreader = csv.reader(csvfile,dialect='excel')
        csvHeadersLS = next(csvreader) # expect and handle 1st row as header row
        for row in csvreader:
            bmp_name = row[csvHeadersLS.index('BMP_Name')] #get BMP name from CSV row
            '''list expression information needed for import
                each element is a list of information for an expression and includes:
                [UnitCSVColumnName, ExpressionName, ExprStrCSVColumnName, statusStatment,
                base_bmp table field name associated w/ this expression]
            '''
            importExprLS = [
                ['CIP_Unit', 'cip_cost_expr_' + bmp_name, 'CIP_Cost_Equation', 'Reading csv cip cost info...', 'cip_expression_id'],
                ['O&M_Unit', 'om_cost_expr_' + bmp_name, 'O&M_Cost_Equation', 'Reading csv o&m cost info...', 'om_expression_id'], 
                ['BMP_SIZE_Unit', 'bmp_size_expr_' + bmp_name, 'BMP_SIZE_Equation', 'Reading csv bmp sizing info...', 'bmp_size_expression_id']                
                ] 
            #use dict. comprehension to represent association between the _HELPER_ImportBaseBMPsCSV returned record_id
            #and a foreign key field name. Dictionary is of form: {foreign_key_field_name:record_id}   
            print ('\nReading csv record: ' + bmp_name)         
            cip_om_sizeDict = {element[4]: _HELPER_ImportBaseBMPsCSV_Expr(element, row, csvHeadersLS) for element in importExprLS}

            #write pollutant removal rates
            print ('Reading pollutant removal rate info...')
            pollLS = ['r_tss','r_turbidity','r_og','r_fe','r_cu','r_zn','r_n','r_p','r_nn','r_an','r_phmax'] #list of pollutants corresponding to csv headr names
            myRecDict = {rp:row[csvHeadersLS.index(rp)] for rp in pollLS} #create dictionary of pollutant values by looking up index of header corresponding to pollutant.
            myTable = Base.metadata.tables['pollutant_removal_rates']
            PRR_id= SQLA_main.insertupdateRec(myTable, myRecDict,(lambda expr_nameCol, expr_Val: expr_nameCol == expr_Val)(myTable.c['id'], -1234))

            #we now have information needed to create base_bmp record
            tmpDict = {'bmp_name':bmp_name, 'bmp_removal_rates_id':PRR_id}
            myRecDict = {**tmpDict, **cip_om_sizeDict}
            myTable = Base.metadata.tables['base_bmps']
            mybase_bmp_id = SQLA_main.insertupdateRec(myTable, myRecDict,(lambda x,y: x == y)(myTable.c['bmp_name'], bmp_name))

        #use base bmp record to create bmp_feasibility_test_definition:
            #check that we have base_bmp record:
            if mybase_bmp_id > 0:
                print ('Linking feasibility tests w/ base bmp: ' + str(mybase_bmp_id))
                myTable = Base.metadata.tables['base_bmp_feasibility_test_definitions']
                #'DELETE old feasibility test definitions for the base bmp
                result = session.query(myTable).filter(myTable.c['base_bmp_id'] == mybase_bmp_id).delete(synchronize_session = False)
                print ('Removed: ', result, ' old feasibility test defs for the base bmp')
                #if feas test at csv column is not N/A then add record 
                for csvCol in range(C_feastest_StartCol, len(row)):
                    if row[csvCol] != 'N/A':
                        ftq_id = session.query(Feasibility_Test_Questions.id).filter(Feasibility_Test_Questions.feas_id == csvHeadersLS[csvCol]).first()
                        if ftq_id is None:
                            print ('!!!!WARNING!!!! No record for feas_id: ', csvHeadersLS[csvCol], ' in Feasibility_Test_Questions table. Skipping...')
                        else:                           
                            myRecDict = {'feasibility_test_question_id':ftq_id[0], 'base_bmp_id':mybase_bmp_id}
                            myRecID = SQLA_main.insertupdateRec(myTable,myRecDict,(lambda x,y: x == y)(myTable.c['id'],-1234))
                            print ('Added feasibility test def as record: ', myRecID)

importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv')    
session.commit()

Reading csv for import to base bmp tables

Reading csv record: Hydrodynamic Separation
Adding to variable dictionary: WQFR
Adding to variable dictionary: WQFR
Reading pollutant removal rate info...
Linking feasibility tests w/ base bmp: 1
Removed:  0  old feasibility test defs for the base bmp
Added feasibility test def as record:  1
Added feasibility test def as record:  2
Added feasibility test def as record:  3
Added feasibility test def as record:  4

Reading csv record: Enhanced Media Filtration (Replaceable Cartridge)
Adding to variable dictionary: WQFR
Adding to variable dictionary: WQFR
Adding to variable dictionary: WQFR
Reading pollutant removal rate info...
Linking feasibility tests w/ base bmp: 2
Removed:  0  old feasibility test defs for the base bmp
Added feasibility test def as record:  5
Added feasibility test def as record:  6
Added feasibility test def as record:  7
Added feasibility test def as record:  8

Reading csv record: Biofiltration (Vault)
Adding to variable 

In [6]:
DICT1 = {'a':1, 'b':2}
DICT2 = {'c':3, 'd':4}
DICT3 = dict(DICT1, **{'a':1, 'b':2})
print (DICT3)

print (DICT1)

print (DICT2)
z = {**DICT1, **DICT2}
dict()

{'a': 1, 'b': 2}
{'a': 1, 'b': 2}
{'c': 3, 'd': 4}


{}

In [7]:
myTable = Base.metadata.tables['base_bmps']
# lambda expr_nameCol, expr_Val: expr_nameCol == expr_Val)(myTable.c['bmp_name'],bmp_name)
ret = session.query(myTable.c['bmp_name']).filter((lambda expr_nameCol, expr_Val: expr_nameCol == expr_Val)(myTable.c['bmp_name'],'Hydrodynamic Separation')) #determine existance of record w/ whereConstraint)) #determine existance of record w/ whereConstraint
print (ret.one())
#                                                                                                             

('Hydrodynamic Separation',)


In [10]:
session.close()
engine.dispose()

In [9]:
def f1(passedFunc):
    print (passedFunc)
def main():
    z = 10
    
    f1((lambda x, y: x * y)(3,z))
#     f1((lambda x: x **2)(3))

main()
    

30
